## Final Project Submission

Please fill out:
* Student name: Gregory Antony Mikuro
* Student pace: Full Time
* Scheduled project review date/time: 22/03/2024
* Instructor name: Nikita Njoroge
* Blog post URL: 


In [570]:
# Your code here - remember to use markdown cells for comments as well!

# Import Statements

In [571]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile 
import sqlite3

%matplotlib inline

## 1. Data Preparation - Extraction and Cleaning 

### 1.1 Box Office Mojo 

In [572]:
# Read the compressed CSV file
bom_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')

# Display the contents of the DataFrame
bom_df.head()


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [573]:
# note: Display the last 5 rows of the 'bom_df' DataFrame
bom_df.tail()

,title,studio,domestic_gross,foreign_gross,year
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
3386,An Actor Prepares,Grav.,1700.0,NaN,2018


In [574]:
# Check the information and missing values in the DataFrame
bom_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [575]:

# Convert the 'foreign_gross' column to string
bom_df['foreign_gross'] = bom_df['foreign_gross'].astype(str)

# Remove the comma from the 'foreign_gross' column before converting it to float
bom_df['foreign_gross'] = bom_df['foreign_gross'].str.replace(',', '').astype(float)



In [576]:
# Check the information and missing values in the DataFrame
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


In [577]:
cleaned_bom_df = bom_df.copy()

# Drop rows with missing values in the 'title' column
cleaned_bom_df = cleaned_bom_df.dropna(subset=['title'])

# Drop rows with missing values in the 'studio' column
cleaned_bom_df = cleaned_bom_df.dropna(subset=['studio'])

# Fill missing values in the 'domestic_gross' column with the median value
cleaned_bom_df['domestic_gross'].fillna(cleaned_bom_df['domestic_gross'].median(), inplace=True)

# Fill missing values in the 'foreign_gross' column with the median value
cleaned_bom_df['foreign_gross'].fillna(cleaned_bom_df['foreign_gross'].median(), inplace=True)

# Convert the 'domestic_gross' column to integer data type
cleaned_bom_df['domestic_gross'] = cleaned_bom_df['domestic_gross'].astype(int)

# Convert the 'foreign_gross' column to integer data type
cleaned_bom_df['foreign_gross'] = cleaned_bom_df['foreign_gross'].astype(int)

# Reset the index of the DataFrame
cleaned_bom_df.reset_index(drop=True, inplace=True)

# Display the first 5 rows of the cleaned DataFrame
cleaned_bom_df



,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000,652000000,2010
1,Alice in Wonderland (2010),BV,334200000,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000,664300000,2010
3,Inception,WB,292600000,535700000,2010
4,Shrek Forever After,P/DW,238700000,513900000,2010
...,...,...,...,...,...
3377,The Quake,Magn.,6200,18700000,2018
3378,Edward II (2018 re-release),FM,4800,18700000,2018
3379,El Pacto,Sony,2500,18700000,2018
3380,The Swan,Synergetic,2400,18700000,2018


In [578]:

# Check the information and missing values in the DataFrame
cleaned_bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3382 entries, 0 to 3381
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           3382 non-null   object
 1   studio          3382 non-null   object
 2   domestic_gross  3382 non-null   int32 
 3   foreign_gross   3382 non-null   int32 
 4   year            3382 non-null   int64 
dtypes: int32(2), int64(1), object(2)
memory usage: 105.8+ KB


### 1.2 IMDB

In [579]:

# Extract the IMDB database file from the zip archive
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('zippedData')



In [580]:
# Connect to the SQLite database file
con = sqlite3.connect('zippedData/im.db')

# Define the SQL query to retrieve all tables in the database
query = """SELECT * FROM sqlite_master"""

# Execute the query and store the result in a DataFrame
tables = pd.read_sql(query, con)

# Display the contents of the DataFrame
tables



,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [581]:
# Define the SQL query to retrieve the movie_ratings table
query_movie_ratings = "SELECT * FROM movie_ratings"

# Execute the query and store the result in a DataFrame
movie_ratings = pd.read_sql(query_movie_ratings, con)

# Display the contents of the DataFrame
movie_ratings




,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [582]:
# Define the SQL query to retrieve the movie_basics table
query_movie_basics = "SELECT * FROM movie_basics"


# Execute the query and store the result in a DataFrame
movie_basics = pd.read_sql(query_movie_basics, con)

# Display the contents of the DataFrame
movie_basics


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [583]:
# Define the SQL query
query_movie_basics = "SELECT * FROM movie_basics"

# Execute the query and store the results in a DataFrame
movie_basics = pd.read_sql_query(query_movie_basics, con)

# Display the information about the DataFrame
movie_basics.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [584]:
# Merge movie_ratings and movie_basics on 'movie_id' column
combined_im_df = movie_ratings.merge(movie_basics, on='movie_id')
combined_im_df

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,Borderless,Borderless,2019,87.0,Documentary
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"
...,...,...,...,...,...,...,...,...
73851,tt9805820,8.1,25,Caisa,Caisa,2018,84.0,Documentary
73852,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120.0,"Action,Animation,Sci-Fi"
73853,tt9851050,4.7,14,Sisters,Sisters,2019,NaN,"Action,Drama"
73854,tt9886934,7.0,5,The Projectionist,The Projectionist,2019,81.0,Documentary


In [585]:
def get_combined_im_info():
    """
    This function prints the information about the combined_im_df DataFrame.

    Note: The combined_im_df DataFrame contains data related to box office insights.
    """
    combined_im_df.info()

get_combined_im_info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   averagerating    73856 non-null  float64
 2   numvotes         73856 non-null  int64  
 3   primary_title    73856 non-null  object 
 4   original_title   73856 non-null  object 
 5   start_year       73856 non-null  int64  
 6   runtime_minutes  66236 non-null  float64
 7   genres           73052 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 5.1+ MB


In [586]:
# Drop rows with missing values in the 'runtime_minutes' column
combined_im_df = combined_im_df.dropna(subset=['runtime_minutes']).copy()

# Fill missing values in the 'genres' column with a default value
combined_im_df.loc[:, 'genres'].fillna('Unknown', inplace=True)

# Convert the 'start_year' column to string data type
combined_im_df.loc[:, 'start_year'] = combined_im_df['start_year'].astype(str)

# Convert the 'runtime_minutes' column to integer data type
combined_im_df.loc[:, 'runtime_minutes'] = combined_im_df['runtime_minutes'].astype(int)

# Reset the index of the DataFrame
combined_im_df.reset_index(drop=True, inplace=True)

combined_im_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66236 entries, 0 to 66235
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         66236 non-null  object 
 1   averagerating    66236 non-null  float64
 2   numvotes         66236 non-null  int64  
 3   primary_title    66236 non-null  object 
 4   original_title   66236 non-null  object 
 5   start_year       66236 non-null  object 
 6   runtime_minutes  66236 non-null  int32  
 7   genres           66236 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 3.8+ MB


In [587]:
def my_function():
    """
    This function returns the combined_im_df dataframe.

    Returns:
    combined_im_df (pandas.DataFrame): The combined dataframe containing box office insights.
    """
    return combined_im_df
    
combined_im_df

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117,Romance
1,tt10384606,8.9,559,Borderless,Borderless,2019,87,Documentary
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90,Drama
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73,"Mystery,Thriller"
...,...,...,...,...,...,...,...,...
66231,tt9768966,8.6,27,Plugged in,Plugged in,2019,53,Documentary
66232,tt9805820,8.1,25,Caisa,Caisa,2018,84,Documentary
66233,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120,"Action,Animation,Sci-Fi"
66234,tt9886934,7.0,5,The Projectionist,The Projectionist,2019,81,Documentary


In [588]:
# Check if the 'genres' column exists in the DataFrame
if 'genres' in combined_im_df.columns:
    # Split the 'genres' column into multiple columns
    combined_im_df[['Genre1', 'Genre2', 'Genre3']] = combined_im_df['genres'].str.split(',', expand=True)

# Display the updated DataFrame
combined_im_df

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,Genre1,Genre2,Genre3
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117,Romance,Romance,None,None
1,tt10384606,8.9,559,Borderless,Borderless,2019,87,Documentary,Documentary,None,None
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90,Drama,Drama,None,None
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99,"Action,Adventure,Fantasy",Action,Adventure,Fantasy
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73,"Mystery,Thriller",Mystery,Thriller,None
...,...,...,...,...,...,...,...,...,...,...,...
66231,tt9768966,8.6,27,Plugged in,Plugged in,2019,53,Documentary,Documentary,None,None
66232,tt9805820,8.1,25,Caisa,Caisa,2018,84,Documentary,Documentary,None,None
66233,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120,"Action,Animation,Sci-Fi",Action,Animation,Sci-Fi
66234,tt9886934,7.0,5,The Projectionist,The Projectionist,2019,81,Documentary,Documentary,None,None


In [589]:
# This code calls the `info()` method on the `combined_im_df` DataFrame to retrieve information about its structure and data types.

combined_im_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66236 entries, 0 to 66235
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         66236 non-null  object 
 1   averagerating    66236 non-null  float64
 2   numvotes         66236 non-null  int64  
 3   primary_title    66236 non-null  object 
 4   original_title   66236 non-null  object 
 5   start_year       66236 non-null  object 
 6   runtime_minutes  66236 non-null  int32  
 7   genres           66236 non-null  object 
 8   Genre1           66236 non-null  object 
 9   Genre2           34877 non-null  object 
 10  Genre3           17840 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(8)
memory usage: 5.3+ MB


In [590]:
# Define the columns to drop from the DataFrame
columns_to_drop = ['Genre2', 'Genre3']

# Check if the columns exist in the DataFrame
columns_to_drop = [col for col in columns_to_drop if col in combined_im_df.columns]

# Drop the columns from the DataFrame
combined_im_df.drop(columns_to_drop, axis=1, inplace=True)

# Display the updated DataFrame
combined_im_df

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,Genre1
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117,Romance,Romance
1,tt10384606,8.9,559,Borderless,Borderless,2019,87,Documentary,Documentary
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90,Drama,Drama
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99,"Action,Adventure,Fantasy",Action
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73,"Mystery,Thriller",Mystery
...,...,...,...,...,...,...,...,...,...
66231,tt9768966,8.6,27,Plugged in,Plugged in,2019,53,Documentary,Documentary
66232,tt9805820,8.1,25,Caisa,Caisa,2018,84,Documentary,Documentary
66233,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120,"Action,Animation,Sci-Fi",Action
66234,tt9886934,7.0,5,The Projectionist,The Projectionist,2019,81,Documentary,Documentary


In [591]:
# This code calls the `info()` method on the `combined_im_df` DataFrame to retrieve information about its structure and data types.

combined_im_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66236 entries, 0 to 66235
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         66236 non-null  object 
 1   averagerating    66236 non-null  float64
 2   numvotes         66236 non-null  int64  
 3   primary_title    66236 non-null  object 
 4   original_title   66236 non-null  object 
 5   start_year       66236 non-null  object 
 6   runtime_minutes  66236 non-null  int32  
 7   genres           66236 non-null  object 
 8   Genre1           66236 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(6)
memory usage: 4.3+ MB


### 1.3 TheMovieDB

In [592]:
# Read the compressed CSV file
tmdb_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz')

# Display the contents of the DataFrame
tmdb_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [593]:
# Check the information and missing values in the DataFrame
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [594]:

# Extract genre_ids from tmdb_df and convert them to a list of integers
genre_ids_list = []
for row in tmdb_df['genre_ids']:
    if row == '[]':
        genre_ids_list.append([])
    else:
        genre_ids_list.append([int(genre_id) for genre_id in row.strip('[]').split(', ')])


# Extract the primary genres from genre_ids_list
primary_genres = [genre_ids[0] if genre_ids else None for genre_ids in genre_ids_list]

# Create the updated_pk_tmdb_df DataFrame with the new column
updated_pk_tmdb_df = tmdb_df.copy()
updated_pk_tmdb_df['primary_genre'] = primary_genres

# Replace non-finite values with a default value
updated_pk_tmdb_df['primary_genre'] = updated_pk_tmdb_df['primary_genre'].fillna(-1)

# Convert the column to integers
updated_pk_tmdb_df['primary_genre'] = updated_pk_tmdb_df['primary_genre'].astype(int)

#show the updated DataFrame
updated_pk_tmdb_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,primary_genre
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,12
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,14
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,12
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,16
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,28


In [595]:

# display the last 5 rows of the 'updated_pk_tmdb_df' DataFrame

updated_pk_tmdb_df.tail()


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,primary_genre
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.6,2018-10-13,Laboratory Conditions,0.0,1,27
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.6,2018-05-01,_EXHIBIT_84xxx_,0.0,1,18
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.6,2018-10-01,The Last One,0.0,1,14
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.6,2018-06-22,Trailer Made,0.0,1,10751
26516,26516,"[53, 27]",309885,en,The Church,0.6,2018-10-05,The Church,0.0,1,53


In [596]:
#Summarize the concise information about the structure and data types of updated_pk_tmdb_df.

updated_pk_tmdb_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
 10  primary_genre      26517 non-null  int32  
dtypes: float64(2), int32(1), int64(3), object(5)
memory usage: 2.1+ MB


The code for the genre IDs was discovered on a forum discussion page hosted on themoviedb.org. The forum discussion, located at the provided URL, likely contained a conversation or thread where users shared information about how to access and utilize genre IDs within the Movie Database API. These genre IDs are essential for categorizing movies into different genres and are commonly used in various applications and services that interact with movie databases, including analysis, recommendation systems, and more.

https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee

In [597]:
# Define dictionary mapping integer values to genre names
backwards_key = {
    '28': 'Action',
    '12': 'Adventure',
    '16': 'Animation',
    '35': 'Comedy',
    '80': 'Crime',
    '99': 'Documentary',
    '18': 'Drama',
    '10751': 'Family',
    '14': 'Fantasy',
    '36': 'History',
    '27': 'Horror',
    '10402': 'Music',
    '9648': 'Mystery',
    '10749': 'Romance',
    '878': 'Science Fiction',
    '10770': 'TV Movie',
    '53': 'Thriller',
    '10752': 'War',
    '37': 'Western'
}

# Replace integer values in 'primary_genre' column with genre names
updated_pk_tmdb_df['primary_genre_name'] = updated_pk_tmdb_df['primary_genre'].astype(str).replace(backwards_key)

# Display the updated DataFrame
updated_pk_tmdb_df.head()


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,primary_genre,primary_genre_name
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,12,Adventure
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,14,Fantasy
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,12,Adventure
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,16,Animation
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,28,Action


In [598]:
## Summarize essential information about the structure and data types of updated_pk_tmdb_df.

updated_pk_tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          26517 non-null  int64  
 1   genre_ids           26517 non-null  object 
 2   id                  26517 non-null  int64  
 3   original_language   26517 non-null  object 
 4   original_title      26517 non-null  object 
 5   popularity          26517 non-null  float64
 6   release_date        26517 non-null  object 
 7   title               26517 non-null  object 
 8   vote_average        26517 non-null  float64
 9   vote_count          26517 non-null  int64  
 10  primary_genre       26517 non-null  int32  
 11  primary_genre_name  26517 non-null  object 
dtypes: float64(2), int32(1), int64(3), object(6)
memory usage: 2.3+ MB


In [599]:
## Display the last few rows of the DataFrame updated_pk_tmdb_df.

updated_pk_tmdb_df.tail()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,primary_genre,primary_genre_name
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.6,2018-10-13,Laboratory Conditions,0.0,1,27,Horror
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.6,2018-05-01,_EXHIBIT_84xxx_,0.0,1,18,Drama
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.6,2018-10-01,The Last One,0.0,1,14,Fantasy
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.6,2018-06-22,Trailer Made,0.0,1,10751,Family
26516,26516,"[53, 27]",309885,en,The Church,0.6,2018-10-05,The Church,0.0,1,53,Thriller


### 1.4 The Numbers

In [600]:
# Read the compressed CSV file
tn_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')

# Display the contents of the DataFrame
tn_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [601]:
# Check the information and missing values in the DataFrame
tn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [602]:

# Convert the columns to string type
tn_df['production_budget'] = tn_df['production_budget'].astype(str)
tn_df['domestic_gross'] = tn_df['domestic_gross'].astype(str)
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].astype(str)

# Remove the '$' and ',' characters and convert the columns to int64 type
tn_df['production_budget'] = tn_df['production_budget'].str.replace('$', '').str.replace(',', '').astype('int64')
tn_df['domestic_gross'] = tn_df['domestic_gross'].str.replace('$', '').str.replace(',', '').astype('int64')
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].str.replace('$', '').str.replace(',', '').astype('int64')


In [603]:
#Display concise summary information about the DataFrame tn_df.


tn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   int64 
 5   worldwide_gross    5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 271.2+ KB
